In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

2024-07-30 12:40:58.954179: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-30 12:40:59.521572: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 12:40:59.521622: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 12:40:59.684642: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-30 12:40:59.887931: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# load the trained model
gmp_model = tf.keras.models.load_model('../models/trained_gmp_model_dense_32_new.h5')

2024-07-30 12:41:17.351168: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [3]:
gmp_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                           

In [4]:
# feature extractor input
feat_ext_input = gmp_model.input

In [5]:
# feature extractor output - do this at the ReLu activation layer - as this will give the same features as the dropout layer (It does not matter if it is the dropout or the activation layer, the extracted features will be the same)
feat_ext_output = gmp_model.layers[-4].output

In [6]:
feature_extractor_model = tf.keras.models.Model(inputs = feat_ext_input, outputs = feat_ext_output)

In [7]:
feature_extractor_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                             

In [8]:
# We might need to be cautious here though, as dropout will set some neurons to 0, and the extracted features will therefore become zero - workaround - use the Dense layer to extract features before the pred head (and the dropout layer)

Locate the previously preprocessed data

In [ ]:
# We currently have just a single block - 0103

In [ ]:
prev_preprocessed_image_loc = "preprocessed_data/Block_0103/sub_images_and_counts"

In [ ]:
folder_contents = os.listdir(prev_preprocessed_image_loc)
folder_contents.sort()

In [ ]:
len(folder_contents)/(2*42)

In [ ]:
folder_contents[0]

In [ ]:
try_fold = np.load(os.path.join(prev_preprocessed_image_loc, folder_contents[0]))
try_fold.shape

In [ ]:
try_fold = try_fold[None,  ...]

In [ ]:
# see if the model gives predictions
pred_feats_try = feature_extractor_model.predict(try_fold)

In [ ]:
pred_feats_try.shape

In [ ]:
pred_feats_try

In [ ]:
# may need to write a function that does feature extraction as well as get the respective density values for each subwindow - this may have to be carried manually to each sub window, but is there a way to automate this? Think on this

In [ ]:
# sub 10
digits_im = 3
digits_d = 11
character = '10'
sub_10_images = [item for item in folder_contents if item.split(".")[0][-digits_im:] == ('_' + character)]
sub_10_dense_maps = [item for item in folder_contents if item.split(".")[0][-digits_d:] == ('_' + character + '_density')]

In [ ]:
len(sub_10_images)

In [ ]:
sub_10_images

In [ ]:
len(sub_10_dense_maps)

In [ ]:
sub_10_dense_maps

In [ ]:
all_preds = []
all_densitites = []
for i in range(len(sub_10_images)):
    # load the subimage
    load_image = np.load(os.path.join(prev_preprocessed_image_loc, sub_10_images[i]))
    # reshape it to get the prediction
    load_image_reshape = load_image[None, ...]
    # get the prediction from the model
    extracted_features = feature_extractor_model.predict(load_image_reshape)
    all_preds.append(extracted_features)

    # load the count map
    load_count_map = np.load(os.path.join(prev_preprocessed_image_loc, sub_10_dense_maps[i]))
    count = np.sum(load_count_map)
    all_densitites.append(count)

In [ ]:
len(all_preds)

In [9]:
def extract_features_and_get_counts(model, preprocessed_path, all_in_folder, digits_image, digits_dense, character, store_path):
    necessary_sub_images = [item for item in all_in_folder if item.split(".")[0][-digits_image:] == ('_' + character)]
    necessary_sub_images.sort()
    necessary_sub_dense_maps = [item for item in all_in_folder if item.split(".")[0][-digits_dense:] == ('_' + character + '_density')]
    necessary_sub_dense_maps.sort()
    print(necessary_sub_images[0])
    # check if the image and density maps tally
    print(np.mean([item.split('.')[0] for item in necessary_sub_images] == [item.split(".")[0][:-8] for item in necessary_sub_dense_maps]))
    
    collect_extracted_features = []
    collect_corresponding_counts = []

    # collect the subwindow-wise features and corresponding counts
    for i in range(len(necessary_sub_images)):
        # load the subimage
        load_image = np.load(os.path.join(preprocessed_path, necessary_sub_images[i]))
        # reshape it to get the prediction
        load_image_reshape = load_image[None, ...]
        # get the prediction from the model
        extracted_features = model.predict(load_image_reshape, verbose = 0)
        collect_extracted_features.append(extracted_features)

        # load the count map
        load_count_map = np.load(os.path.join(preprocessed_path, necessary_sub_dense_maps[i]))
        count = np.sum(load_count_map)
        collect_corresponding_counts.append(count)

    # collect all extracted features data and create a dataframe
    Features_data = pd.DataFrame(np.vstack(collect_extracted_features), columns = ['feature_' + str(i) for i in range(np.vstack(collect_extracted_features).shape[1])])
    # dataframe for counts
    counts_subwindow = pd.DataFrame(collect_corresponding_counts, columns = ['tassel_count'])
    # combine the two dataframes
    combined_df = pd.concat((Features_data,counts_subwindow), axis = 1 )
    # save this dataframe
    save_path_name = store_path + 'extracted_features_sub_window_' + character + '.csv'
    combined_df.to_csv(save_path_name, index = False)

    return(combined_df)

In [ ]:
s_path = 'preprocessed_data/Block_0103/TS_ready_data_frames/'

In [ ]:
# try the function
hello_1 = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc, folder_contents, 2, 10, '1', s_path)

In [ ]:
hello_1.head()

In [ ]:
hello_1.shape

In [ ]:
# try the function for another character
hello_2 = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc, folder_contents, 2, 10, '2', s_path)

In [ ]:
hello_2.head()

In [ ]:
hello_2.shape

In [ ]:
# get the predictions for all subimages for the block? do separately for single digit subimages and double digit subimages separately
character_list = list(range(10))
character_list = list(map(str, character_list))

In [ ]:
%%time

# for single digit subwindows
catch_all_dfs = []
for character in character_list:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc, folder_contents, 2, 10, character, s_path)
    catch_all_dfs.append(features_df)

In [ ]:
# get the predictions for double digits
character_list_double = list(range(10, 42))
character_list_double = list(map(str, character_list_double))

In [ ]:
# character_list_double

In [ ]:
%%time

# for double digit subwindows
catch_all_dfs = []
for character in character_list_double:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc, folder_contents, 3, 11, character, s_path)
    catch_all_dfs.append(features_df)

Block 0104

In [10]:
prev_preprocessed_image_loc_0104 = "preprocessed_data/Block_0104/sub_images_and_counts"

In [11]:
folder_contents_0104 = os.listdir(prev_preprocessed_image_loc_0104)
folder_contents_0104.sort()

In [12]:
# folder_contents

In [13]:
folder_contents_0104[0]

'Block0104_2020_08_03_0.npy'

In [14]:
s_path_0104 = 'preprocessed_data/Block_0104//TS_ready_data_frames/'

In [15]:
# get the predictions for all subimages for the block? do separately for single digit subimages and double digit subimages separately
character_list = list(range(10))
character_list = list(map(str, character_list))

In [16]:
%%time

# for single digit subwindows
catch_all_dfs_0104 = []
for character in character_list:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0104, folder_contents_0104, 2, 10, character, s_path_0104)
    catch_all_dfs_0104.append(features_df)

Block0104_2020_08_03_0.npy
1.0


2024-07-30 12:41:19.435512: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


Block0104_2020_08_03_1.npy
1.0
Block0104_2020_08_03_2.npy
1.0
Block0104_2020_08_03_3.npy
1.0
Block0104_2020_08_03_4.npy
1.0
Block0104_2020_08_03_5.npy
1.0
Block0104_2020_08_03_6.npy
1.0
Block0104_2020_08_03_7.npy
1.0
Block0104_2020_08_03_8.npy
1.0
Block0104_2020_08_03_9.npy
1.0
CPU times: user 11.8 s, sys: 1.2 s, total: 13 s
Wall time: 39.7 s


In [17]:
# get the predictions for double digits
character_list_double = list(range(10, 42))
character_list_double = list(map(str, character_list_double))

In [18]:
%%time

# for double digit subwindows
catch_all_dfs_0104 = []
for character in character_list_double:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0104, folder_contents_0104, 3, 11, character, s_path_0104)
    catch_all_dfs_0104.append(features_df)

Block0104_2020_08_03_10.npy
1.0
Block0104_2020_08_03_11.npy
1.0
Block0104_2020_08_03_12.npy
1.0
Block0104_2020_08_03_13.npy
1.0
Block0104_2020_08_03_14.npy
1.0
Block0104_2020_08_03_15.npy
1.0
Block0104_2020_08_03_16.npy
1.0
Block0104_2020_08_03_17.npy
1.0
Block0104_2020_08_03_18.npy
1.0
Block0104_2020_08_03_19.npy
1.0
Block0104_2020_08_03_20.npy
1.0
Block0104_2020_08_03_21.npy
1.0
Block0104_2020_08_03_22.npy
1.0
Block0104_2020_08_03_23.npy
1.0
Block0104_2020_08_03_24.npy
1.0
Block0104_2020_08_03_25.npy
1.0
Block0104_2020_08_03_26.npy
1.0
Block0104_2020_08_03_27.npy
1.0
Block0104_2020_08_03_28.npy
1.0
Block0104_2020_08_03_29.npy
1.0
Block0104_2020_08_03_30.npy
1.0
Block0104_2020_08_03_31.npy
1.0
Block0104_2020_08_03_32.npy
1.0
Block0104_2020_08_03_33.npy
1.0
Block0104_2020_08_03_34.npy
1.0
Block0104_2020_08_03_35.npy
1.0
Block0104_2020_08_03_36.npy
1.0
Block0104_2020_08_03_37.npy
1.0
Block0104_2020_08_03_38.npy
1.0
Block0104_2020_08_03_39.npy
1.0
Block0104_2020_08_03_40.npy
1.0
Block010

Block 0105

In [19]:
prev_preprocessed_image_loc_0105 = "preprocessed_data/Block_0105/sub_images_and_counts"

In [20]:
folder_contents_0105 = os.listdir(prev_preprocessed_image_loc_0105)
folder_contents_0105.sort()

In [21]:
# folder_contents

In [22]:
folder_contents_0105[0]

'Block0105_2020_08_03_0.npy'

In [23]:
s_path_0105 = 'preprocessed_data/Block_0105//TS_ready_data_frames/'

In [24]:
# get the predictions for all subimages for the block? do separately for single digit subimages and double digit subimages separately
character_list = list(range(10))
character_list = list(map(str, character_list))

In [25]:
%%time

# for single digit subwindows
catch_all_dfs_0105 = []
for character in character_list:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0105, folder_contents_0105, 2, 10, character, s_path_0105)
    catch_all_dfs_0105.append(features_df)

Block0105_2020_08_03_0.npy
1.0
Block0105_2020_08_03_1.npy
1.0
Block0105_2020_08_03_2.npy
1.0
Block0105_2020_08_03_3.npy
1.0
Block0105_2020_08_03_4.npy
1.0
Block0105_2020_08_03_5.npy
1.0
Block0105_2020_08_03_6.npy
1.0
Block0105_2020_08_03_7.npy
1.0
Block0105_2020_08_03_8.npy
1.0
Block0105_2020_08_03_9.npy
1.0
CPU times: user 10.5 s, sys: 972 ms, total: 11.5 s
Wall time: 15.1 s


In [26]:
# get the predictions for double digits
character_list_double = list(range(10, 42))
character_list_double = list(map(str, character_list_double))

In [27]:
%%time

# for double digit subwindows
catch_all_dfs_0105 = []
for character in character_list_double:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0105, folder_contents_0105, 3, 11, character, s_path_0105)
    catch_all_dfs_0105.append(features_df)

Block0105_2020_08_03_10.npy
1.0
Block0105_2020_08_03_11.npy
1.0
Block0105_2020_08_03_12.npy
1.0
Block0105_2020_08_03_13.npy
1.0
Block0105_2020_08_03_14.npy
1.0
Block0105_2020_08_03_15.npy
1.0
Block0105_2020_08_03_16.npy
1.0
Block0105_2020_08_03_17.npy
1.0
Block0105_2020_08_03_18.npy
1.0
Block0105_2020_08_03_19.npy
1.0
Block0105_2020_08_03_20.npy
1.0
Block0105_2020_08_03_21.npy
1.0
Block0105_2020_08_03_22.npy
1.0
Block0105_2020_08_03_23.npy
1.0
Block0105_2020_08_03_24.npy
1.0
Block0105_2020_08_03_25.npy
1.0
Block0105_2020_08_03_26.npy
1.0
Block0105_2020_08_03_27.npy
1.0
Block0105_2020_08_03_28.npy
1.0
Block0105_2020_08_03_29.npy
1.0
Block0105_2020_08_03_30.npy
1.0
Block0105_2020_08_03_31.npy
1.0
Block0105_2020_08_03_32.npy
1.0
Block0105_2020_08_03_33.npy
1.0
Block0105_2020_08_03_34.npy
1.0
Block0105_2020_08_03_35.npy
1.0
Block0105_2020_08_03_36.npy
1.0
Block0105_2020_08_03_37.npy
1.0
Block0105_2020_08_03_38.npy
1.0
Block0105_2020_08_03_39.npy
1.0
Block0105_2020_08_03_40.npy
1.0
Block010

In [ ]:
# catch_all_dfs_0105

Block 0106

In [28]:
prev_preprocessed_image_loc_0106 = "preprocessed_data/Block_0106/sub_images_and_counts"

In [29]:
folder_contents_0106 = os.listdir(prev_preprocessed_image_loc_0106)
folder_contents_0106.sort()

In [30]:
# folder_contents

In [31]:
folder_contents_0106[0]

'Block0106_2020_08_03_0.npy'

In [32]:
s_path_0106 = 'preprocessed_data/Block_0106/TS_ready_data_frames/'

In [33]:
# get the predictions for all subimages for the block? do separately for single digit subimages and double digit subimages separately
character_list = list(range(10))
character_list = list(map(str, character_list))

In [34]:
%%time

# for single digit subwindows
catch_all_dfs_0106 = []
for character in character_list:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0106, folder_contents_0106, 2, 10, character, s_path_0106)
    catch_all_dfs_0106.append(features_df)

Block0106_2020_08_03_0.npy
1.0
Block0106_2020_08_03_1.npy
1.0
Block0106_2020_08_03_2.npy
1.0
Block0106_2020_08_03_3.npy
1.0
Block0106_2020_08_03_4.npy
1.0
Block0106_2020_08_03_5.npy
1.0
Block0106_2020_08_03_6.npy
1.0
Block0106_2020_08_03_7.npy
1.0
Block0106_2020_08_03_8.npy
1.0
Block0106_2020_08_03_9.npy
1.0
CPU times: user 11.5 s, sys: 1.18 s, total: 12.7 s
Wall time: 38.8 s


In [35]:
# get the predictions for double digits
character_list_double = list(range(10, 42))
character_list_double = list(map(str, character_list_double))

In [36]:
%%time

# for double digit subwindows
catch_all_dfs_0106 = []
for character in character_list_double:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0106, folder_contents_0106, 3, 11, character, s_path_0106)
    catch_all_dfs_0106.append(features_df)

Block0106_2020_08_03_10.npy
1.0
Block0106_2020_08_03_11.npy
1.0
Block0106_2020_08_03_12.npy
1.0
Block0106_2020_08_03_13.npy
1.0
Block0106_2020_08_03_14.npy
1.0
Block0106_2020_08_03_15.npy
1.0
Block0106_2020_08_03_16.npy
1.0
Block0106_2020_08_03_17.npy
1.0
Block0106_2020_08_03_18.npy
1.0
Block0106_2020_08_03_19.npy
1.0
Block0106_2020_08_03_20.npy
1.0
Block0106_2020_08_03_21.npy
1.0
Block0106_2020_08_03_22.npy
1.0
Block0106_2020_08_03_23.npy
1.0
Block0106_2020_08_03_24.npy
1.0
Block0106_2020_08_03_25.npy
1.0
Block0106_2020_08_03_26.npy
1.0
Block0106_2020_08_03_27.npy
1.0
Block0106_2020_08_03_28.npy
1.0
Block0106_2020_08_03_29.npy
1.0
Block0106_2020_08_03_30.npy
1.0
Block0106_2020_08_03_31.npy
1.0
Block0106_2020_08_03_32.npy
1.0
Block0106_2020_08_03_33.npy
1.0
Block0106_2020_08_03_34.npy
1.0
Block0106_2020_08_03_35.npy
1.0
Block0106_2020_08_03_36.npy
1.0
Block0106_2020_08_03_37.npy
1.0
Block0106_2020_08_03_38.npy
1.0
Block0106_2020_08_03_39.npy
1.0
Block0106_2020_08_03_40.npy
1.0
Block010

Block 0201

In [37]:
prev_preprocessed_image_loc_0201 = "preprocessed_data/Block_0201/sub_images_and_counts"

In [38]:
folder_contents_0201 = os.listdir(prev_preprocessed_image_loc_0201)
folder_contents_0201.sort()

In [39]:
# folder_contents

In [40]:
folder_contents_0201[0]

'Block0201_2020_08_03_0.npy'

In [41]:
s_path_0201 = 'preprocessed_data/Block_0201/TS_ready_data_frames/'

In [42]:
# get the predictions for all subimages for the block? do separately for single digit subimages and double digit subimages separately
character_list = list(range(10))
character_list = list(map(str, character_list))

In [43]:
%%time

# for single digit subwindows
catch_all_dfs_0201 = []
for character in character_list:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0201, folder_contents_0201, 2, 10, character, s_path_0201)
    catch_all_dfs_0201.append(features_df)

Block0201_2020_08_03_0.npy
1.0
Block0201_2020_08_03_1.npy
1.0
Block0201_2020_08_03_2.npy
1.0
Block0201_2020_08_03_3.npy
1.0
Block0201_2020_08_03_4.npy
1.0
Block0201_2020_08_03_5.npy
1.0
Block0201_2020_08_03_6.npy
1.0
Block0201_2020_08_03_7.npy
1.0
Block0201_2020_08_03_8.npy
1.0
Block0201_2020_08_03_9.npy
1.0
CPU times: user 11.5 s, sys: 1.18 s, total: 12.7 s
Wall time: 30.1 s


In [44]:
# get the predictions for double digits
character_list_double = list(range(10, 42))
character_list_double = list(map(str, character_list_double))

In [45]:
%%time

# for double digit subwindows
catch_all_dfs_0201 = []
for character in character_list_double:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0201, folder_contents_0201, 3, 11, character, s_path_0201)
    catch_all_dfs_0201.append(features_df)

Block0201_2020_08_03_10.npy
1.0
Block0201_2020_08_03_11.npy
1.0
Block0201_2020_08_03_12.npy
1.0
Block0201_2020_08_03_13.npy
1.0
Block0201_2020_08_03_14.npy
1.0
Block0201_2020_08_03_15.npy
1.0
Block0201_2020_08_03_16.npy
1.0
Block0201_2020_08_03_17.npy
1.0
Block0201_2020_08_03_18.npy
1.0
Block0201_2020_08_03_19.npy
1.0
Block0201_2020_08_03_20.npy
1.0
Block0201_2020_08_03_21.npy
1.0
Block0201_2020_08_03_22.npy
1.0
Block0201_2020_08_03_23.npy
1.0
Block0201_2020_08_03_24.npy
1.0
Block0201_2020_08_03_25.npy
1.0
Block0201_2020_08_03_26.npy
1.0
Block0201_2020_08_03_27.npy
1.0
Block0201_2020_08_03_28.npy
1.0
Block0201_2020_08_03_29.npy
1.0
Block0201_2020_08_03_30.npy
1.0
Block0201_2020_08_03_31.npy
1.0
Block0201_2020_08_03_32.npy
1.0
Block0201_2020_08_03_33.npy
1.0
Block0201_2020_08_03_34.npy
1.0
Block0201_2020_08_03_35.npy
1.0
Block0201_2020_08_03_36.npy
1.0
Block0201_2020_08_03_37.npy
1.0
Block0201_2020_08_03_38.npy
1.0
Block0201_2020_08_03_39.npy
1.0
Block0201_2020_08_03_40.npy
1.0
Block020

Block 0202

In [46]:
prev_preprocessed_image_loc_0202 = "preprocessed_data/Block_0202/sub_images_and_counts"

In [47]:
folder_contents_0202 = os.listdir(prev_preprocessed_image_loc_0202)
folder_contents_0202.sort()

In [48]:
# folder_contents

In [49]:
folder_contents_0202[0]

'Block0202_2020_08_03_0.npy'

In [50]:
s_path_0202 = 'preprocessed_data/Block_0202/TS_ready_data_frames/'

In [51]:
# get the predictions for all subimages for the block? do separately for single digit subimages and double digit subimages separately
character_list = list(range(10))
character_list = list(map(str, character_list))

In [52]:
%%time

# for single digit subwindows
catch_all_dfs_0202 = []
for character in character_list:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0202, folder_contents_0202, 2, 10, character, s_path_0202)
    catch_all_dfs_0202.append(features_df)

Block0202_2020_08_03_0.npy
1.0
Block0202_2020_08_03_1.npy
1.0
Block0202_2020_08_03_2.npy
1.0
Block0202_2020_08_03_3.npy
1.0
Block0202_2020_08_03_4.npy
1.0
Block0202_2020_08_03_5.npy
1.0
Block0202_2020_08_03_6.npy
1.0
Block0202_2020_08_03_7.npy
1.0
Block0202_2020_08_03_8.npy
1.0
Block0202_2020_08_03_9.npy
1.0
CPU times: user 11.5 s, sys: 1.09 s, total: 12.6 s
Wall time: 30.7 s


In [53]:
# get the predictions for double digits
character_list_double = list(range(10, 42))
character_list_double = list(map(str, character_list_double))

In [54]:
%%time

# for double digit subwindows
catch_all_dfs_0202 = []
for character in character_list_double:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0202, folder_contents_0202, 3, 11, character, s_path_0202)
    catch_all_dfs_0202.append(features_df)

Block0202_2020_08_03_10.npy
1.0
Block0202_2020_08_03_11.npy
1.0
Block0202_2020_08_03_12.npy
1.0
Block0202_2020_08_03_13.npy
1.0
Block0202_2020_08_03_14.npy
1.0
Block0202_2020_08_03_15.npy
1.0
Block0202_2020_08_03_16.npy
1.0
Block0202_2020_08_03_17.npy
1.0
Block0202_2020_08_03_18.npy
1.0
Block0202_2020_08_03_19.npy
1.0
Block0202_2020_08_03_20.npy
1.0
Block0202_2020_08_03_21.npy
1.0
Block0202_2020_08_03_22.npy
1.0
Block0202_2020_08_03_23.npy
1.0
Block0202_2020_08_03_24.npy
1.0
Block0202_2020_08_03_25.npy
1.0
Block0202_2020_08_03_26.npy
1.0
Block0202_2020_08_03_27.npy
1.0
Block0202_2020_08_03_28.npy
1.0
Block0202_2020_08_03_29.npy
1.0
Block0202_2020_08_03_30.npy
1.0
Block0202_2020_08_03_31.npy
1.0
Block0202_2020_08_03_32.npy
1.0
Block0202_2020_08_03_33.npy
1.0
Block0202_2020_08_03_34.npy
1.0
Block0202_2020_08_03_35.npy
1.0
Block0202_2020_08_03_36.npy
1.0
Block0202_2020_08_03_37.npy
1.0
Block0202_2020_08_03_38.npy
1.0
Block0202_2020_08_03_39.npy
1.0
Block0202_2020_08_03_40.npy
1.0
Block020

Block 0205

In [55]:
prev_preprocessed_image_loc_0205 = "preprocessed_data/Block_0205/sub_images_and_counts"

In [56]:
folder_contents_0205 = os.listdir(prev_preprocessed_image_loc_0205)
folder_contents_0205.sort()

In [57]:
# folder_contents

In [58]:
folder_contents_0205[0]

'Block0205_2020_08_03_0.npy'

In [59]:
s_path_0205 = 'preprocessed_data/Block_0205/TS_ready_data_frames/'

In [60]:
# get the predictions for all subimages for the block? do separately for single digit subimages and double digit subimages separately
character_list = list(range(10))
character_list = list(map(str, character_list))

In [61]:
%%time

# for single digit subwindows
catch_all_dfs_0205 = []
for character in character_list:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0205, folder_contents_0205, 2, 10, character, s_path_0205)
    catch_all_dfs_0205.append(features_df)

Block0205_2020_08_03_0.npy
1.0
Block0205_2020_08_03_1.npy
1.0
Block0205_2020_08_03_2.npy
1.0
Block0205_2020_08_03_3.npy
1.0
Block0205_2020_08_03_4.npy
1.0
Block0205_2020_08_03_5.npy
1.0
Block0205_2020_08_03_6.npy
1.0
Block0205_2020_08_03_7.npy
1.0
Block0205_2020_08_03_8.npy
1.0
Block0205_2020_08_03_9.npy
1.0
CPU times: user 11.6 s, sys: 1.14 s, total: 12.7 s
Wall time: 34.7 s


In [62]:
# get the predictions for double digits
character_list_double = list(range(10, 42))
character_list_double = list(map(str, character_list_double))

In [63]:
%%time

# for double digit subwindows
catch_all_dfs_0205 = []
for character in character_list_double:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0205, folder_contents_0205, 3, 11, character, s_path_0205)
    catch_all_dfs_0205.append(features_df)

Block0205_2020_08_03_10.npy
1.0
Block0205_2020_08_03_11.npy
1.0
Block0205_2020_08_03_12.npy
1.0
Block0205_2020_08_03_13.npy
1.0
Block0205_2020_08_03_14.npy
1.0
Block0205_2020_08_03_15.npy
1.0
Block0205_2020_08_03_16.npy
1.0
Block0205_2020_08_03_17.npy
1.0
Block0205_2020_08_03_18.npy
1.0
Block0205_2020_08_03_19.npy
1.0
Block0205_2020_08_03_20.npy
1.0
Block0205_2020_08_03_21.npy
1.0
Block0205_2020_08_03_22.npy
1.0
Block0205_2020_08_03_23.npy
1.0
Block0205_2020_08_03_24.npy
1.0
Block0205_2020_08_03_25.npy
1.0
Block0205_2020_08_03_26.npy
1.0
Block0205_2020_08_03_27.npy
1.0
Block0205_2020_08_03_28.npy
1.0
Block0205_2020_08_03_29.npy
1.0
Block0205_2020_08_03_30.npy
1.0
Block0205_2020_08_03_31.npy
1.0
Block0205_2020_08_03_32.npy
1.0
Block0205_2020_08_03_33.npy
1.0
Block0205_2020_08_03_34.npy
1.0
Block0205_2020_08_03_35.npy
1.0
Block0205_2020_08_03_36.npy
1.0
Block0205_2020_08_03_37.npy
1.0
Block0205_2020_08_03_38.npy
1.0
Block0205_2020_08_03_39.npy
1.0
Block0205_2020_08_03_40.npy
1.0
Block020

Block 0206

In [64]:
prev_preprocessed_image_loc_0206 = "preprocessed_data/Block_0206/sub_images_and_counts"

In [65]:
folder_contents_0206 = os.listdir(prev_preprocessed_image_loc_0206)
folder_contents_0206.sort()

In [66]:
# folder_contents

In [67]:
folder_contents_0206[0]

'Block0206_2020_08_03_0.npy'

In [68]:
s_path_0206 = 'preprocessed_data/Block_0206/TS_ready_data_frames/'

In [69]:
# get the predictions for all subimages for the block? do separately for single digit subimages and double digit subimages separately
character_list = list(range(10))
character_list = list(map(str, character_list))

In [70]:
%%time

# for single digit subwindows
catch_all_dfs_0206 = []
for character in character_list:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0206, folder_contents_0206, 2, 10, character, s_path_0206)
    catch_all_dfs_0206.append(features_df)

Block0206_2020_08_03_0.npy
1.0
Block0206_2020_08_03_1.npy
1.0
Block0206_2020_08_03_2.npy
1.0
Block0206_2020_08_03_3.npy
1.0
Block0206_2020_08_03_4.npy
1.0
Block0206_2020_08_03_5.npy
1.0
Block0206_2020_08_03_6.npy
1.0
Block0206_2020_08_03_7.npy
1.0
Block0206_2020_08_03_8.npy
1.0
Block0206_2020_08_03_9.npy
1.0
CPU times: user 11.6 s, sys: 1.18 s, total: 12.8 s
Wall time: 27.8 s


In [71]:
# get the predictions for double digits
character_list_double = list(range(10, 42))
character_list_double = list(map(str, character_list_double))

In [72]:
%%time

# for double digit subwindows
catch_all_dfs_0206 = []
for character in character_list_double:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0206, folder_contents_0206, 3, 11, character, s_path_0206)
    catch_all_dfs_0206.append(features_df)

Block0206_2020_08_03_10.npy
1.0
Block0206_2020_08_03_11.npy
1.0
Block0206_2020_08_03_12.npy
1.0
Block0206_2020_08_03_13.npy
1.0
Block0206_2020_08_03_14.npy
1.0
Block0206_2020_08_03_15.npy
1.0
Block0206_2020_08_03_16.npy
1.0
Block0206_2020_08_03_17.npy
1.0
Block0206_2020_08_03_18.npy
1.0
Block0206_2020_08_03_19.npy
1.0
Block0206_2020_08_03_20.npy
1.0
Block0206_2020_08_03_21.npy
1.0
Block0206_2020_08_03_22.npy
1.0
Block0206_2020_08_03_23.npy
1.0
Block0206_2020_08_03_24.npy
1.0
Block0206_2020_08_03_25.npy
1.0
Block0206_2020_08_03_26.npy
1.0
Block0206_2020_08_03_27.npy
1.0
Block0206_2020_08_03_28.npy
1.0
Block0206_2020_08_03_29.npy
1.0
Block0206_2020_08_03_30.npy
1.0
Block0206_2020_08_03_31.npy
1.0
Block0206_2020_08_03_32.npy
1.0
Block0206_2020_08_03_33.npy
1.0
Block0206_2020_08_03_34.npy
1.0
Block0206_2020_08_03_35.npy
1.0
Block0206_2020_08_03_36.npy
1.0
Block0206_2020_08_03_37.npy
1.0
Block0206_2020_08_03_38.npy
1.0
Block0206_2020_08_03_39.npy
1.0
Block0206_2020_08_03_40.npy
1.0
Block020

Block 0302

In [73]:
prev_preprocessed_image_loc_0302 = "preprocessed_data/Block_0302/sub_images_and_counts"

In [74]:
folder_contents_0302 = os.listdir(prev_preprocessed_image_loc_0302)
folder_contents_0302.sort()

In [75]:
# folder_contents

In [76]:
folder_contents_0302[0]

'Block0302_2020_08_03_0.npy'

In [77]:
s_path_0302 = 'preprocessed_data/Block_0302/TS_ready_data_frames/'

In [78]:
# get the predictions for all subimages for the block? do separately for single digit subimages and double digit subimages separately
character_list = list(range(10))
character_list = list(map(str, character_list))

In [79]:
%%time

# for single digit subwindows
catch_all_dfs_0302 = []
for character in character_list:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0302, folder_contents_0302, 2, 10, character, s_path_0302)
    catch_all_dfs_0302.append(features_df)

Block0302_2020_08_03_0.npy
1.0
Block0302_2020_08_03_1.npy
1.0
Block0302_2020_08_03_2.npy
1.0
Block0302_2020_08_03_3.npy
1.0
Block0302_2020_08_03_4.npy
1.0
Block0302_2020_08_03_5.npy
1.0
Block0302_2020_08_03_6.npy
1.0
Block0302_2020_08_03_7.npy
1.0
Block0302_2020_08_03_8.npy
1.0
Block0302_2020_08_03_9.npy
1.0
CPU times: user 11.7 s, sys: 1.18 s, total: 12.9 s
Wall time: 33.3 s


In [80]:
# get the predictions for double digits
character_list_double = list(range(10, 42))
character_list_double = list(map(str, character_list_double))

In [81]:
%%time

# for double digit subwindows
catch_all_dfs_0302 = []
for character in character_list_double:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0302, folder_contents_0302, 3, 11, character, s_path_0302)
    catch_all_dfs_0302.append(features_df)

Block0302_2020_08_03_10.npy
1.0
Block0302_2020_08_03_11.npy
1.0
Block0302_2020_08_03_12.npy
1.0
Block0302_2020_08_03_13.npy
1.0
Block0302_2020_08_03_14.npy
1.0
Block0302_2020_08_03_15.npy
1.0
Block0302_2020_08_03_16.npy
1.0
Block0302_2020_08_03_17.npy
1.0
Block0302_2020_08_03_18.npy
1.0
Block0302_2020_08_03_19.npy
1.0
Block0302_2020_08_03_20.npy
1.0
Block0302_2020_08_03_21.npy
1.0
Block0302_2020_08_03_22.npy
1.0
Block0302_2020_08_03_23.npy
1.0
Block0302_2020_08_03_24.npy
1.0
Block0302_2020_08_03_25.npy
1.0
Block0302_2020_08_03_26.npy
1.0
Block0302_2020_08_03_27.npy
1.0
Block0302_2020_08_03_28.npy
1.0
Block0302_2020_08_03_29.npy
1.0
Block0302_2020_08_03_30.npy
1.0
Block0302_2020_08_03_31.npy
1.0
Block0302_2020_08_03_32.npy
1.0
Block0302_2020_08_03_33.npy
1.0
Block0302_2020_08_03_34.npy
1.0
Block0302_2020_08_03_35.npy
1.0
Block0302_2020_08_03_36.npy
1.0
Block0302_2020_08_03_37.npy
1.0
Block0302_2020_08_03_38.npy
1.0
Block0302_2020_08_03_39.npy
1.0
Block0302_2020_08_03_40.npy
1.0
Block030

Block 0303

In [82]:
prev_preprocessed_image_loc_0303 = "preprocessed_data/Block_0303/sub_images_and_counts"

In [83]:
folder_contents_0303 = os.listdir(prev_preprocessed_image_loc_0303)
folder_contents_0303.sort()

In [84]:
# folder_contents

In [85]:
folder_contents_0303[0]

'Block0303_2020_08_03_0.npy'

In [86]:
s_path_0303 = 'preprocessed_data/Block_0303/TS_ready_data_frames/'

In [87]:
# get the predictions for all subimages for the block? do separately for single digit subimages and double digit subimages separately
character_list = list(range(10))
character_list = list(map(str, character_list))

In [88]:
%%time

# for single digit subwindows
catch_all_dfs_0303 = []
for character in character_list:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0303, folder_contents_0303, 2, 10, character, s_path_0303)
    catch_all_dfs_0303.append(features_df)

Block0303_2020_08_03_0.npy
1.0
Block0303_2020_08_03_1.npy
1.0
Block0303_2020_08_03_2.npy
1.0
Block0303_2020_08_03_3.npy
1.0
Block0303_2020_08_03_4.npy
1.0
Block0303_2020_08_03_5.npy
1.0
Block0303_2020_08_03_6.npy
1.0
Block0303_2020_08_03_7.npy
1.0
Block0303_2020_08_03_8.npy
1.0
Block0303_2020_08_03_9.npy
1.0
CPU times: user 11.8 s, sys: 1.19 s, total: 13 s
Wall time: 35.1 s


In [89]:
# get the predictions for double digits
character_list_double = list(range(10, 42))
character_list_double = list(map(str, character_list_double))

In [90]:
%%time

# for double digit subwindows
catch_all_dfs_0303 = []
for character in character_list_double:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0303, folder_contents_0303, 3, 11, character, s_path_0303)
    catch_all_dfs_0303.append(features_df)

Block0303_2020_08_03_10.npy
1.0
Block0303_2020_08_03_11.npy
1.0
Block0303_2020_08_03_12.npy
1.0
Block0303_2020_08_03_13.npy
1.0
Block0303_2020_08_03_14.npy
1.0
Block0303_2020_08_03_15.npy
1.0
Block0303_2020_08_03_16.npy
1.0
Block0303_2020_08_03_17.npy
1.0
Block0303_2020_08_03_18.npy
1.0
Block0303_2020_08_03_19.npy
1.0
Block0303_2020_08_03_20.npy
1.0
Block0303_2020_08_03_21.npy
1.0
Block0303_2020_08_03_22.npy
1.0
Block0303_2020_08_03_23.npy
1.0
Block0303_2020_08_03_24.npy
1.0
Block0303_2020_08_03_25.npy
1.0
Block0303_2020_08_03_26.npy
1.0
Block0303_2020_08_03_27.npy
1.0
Block0303_2020_08_03_28.npy
1.0
Block0303_2020_08_03_29.npy
1.0
Block0303_2020_08_03_30.npy
1.0
Block0303_2020_08_03_31.npy
1.0
Block0303_2020_08_03_32.npy
1.0
Block0303_2020_08_03_33.npy
1.0
Block0303_2020_08_03_34.npy
1.0
Block0303_2020_08_03_35.npy
1.0
Block0303_2020_08_03_36.npy
1.0
Block0303_2020_08_03_37.npy
1.0
Block0303_2020_08_03_38.npy
1.0
Block0303_2020_08_03_39.npy
1.0
Block0303_2020_08_03_40.npy
1.0
Block030

Block 0304

In [91]:
prev_preprocessed_image_loc_0304 = "preprocessed_data/Block_0304/sub_images_and_counts"

In [92]:
folder_contents_0304 = os.listdir(prev_preprocessed_image_loc_0304)
folder_contents_0304.sort()

In [93]:
# folder_contents

In [94]:
folder_contents_0304[0]

'Block0304_2020_08_03_0.npy'

In [95]:
s_path_0304 = 'preprocessed_data/Block_0304/TS_ready_data_frames/'

In [96]:
# get the predictions for all subimages for the block? do separately for single digit subimages and double digit subimages separately
character_list = list(range(10))
character_list = list(map(str, character_list))

In [97]:
%%time

# for single digit subwindows
catch_all_dfs_0304 = []
for character in character_list:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0304, folder_contents_0304, 2, 10, character, s_path_0304)
    catch_all_dfs_0304.append(features_df)

Block0304_2020_08_03_0.npy
1.0
Block0304_2020_08_03_1.npy
1.0
Block0304_2020_08_03_2.npy
1.0
Block0304_2020_08_03_3.npy
1.0
Block0304_2020_08_03_4.npy
1.0
Block0304_2020_08_03_5.npy
1.0
Block0304_2020_08_03_6.npy
1.0
Block0304_2020_08_03_7.npy
1.0
Block0304_2020_08_03_8.npy
1.0
Block0304_2020_08_03_9.npy
1.0
CPU times: user 11.8 s, sys: 1.16 s, total: 12.9 s
Wall time: 29.7 s


In [98]:
# get the predictions for double digits
character_list_double = list(range(10, 42))
character_list_double = list(map(str, character_list_double))

In [99]:
%%time

# for double digit subwindows
catch_all_dfs_0304 = []
for character in character_list_double:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0304, folder_contents_0304, 3, 11, character, s_path_0304)
    catch_all_dfs_0304.append(features_df)

Block0304_2020_08_03_10.npy
1.0
Block0304_2020_08_03_11.npy
1.0
Block0304_2020_08_03_12.npy
1.0
Block0304_2020_08_03_13.npy
1.0
Block0304_2020_08_03_14.npy
1.0
Block0304_2020_08_03_15.npy
1.0
Block0304_2020_08_03_16.npy
1.0
Block0304_2020_08_03_17.npy
1.0
Block0304_2020_08_03_18.npy
1.0
Block0304_2020_08_03_19.npy
1.0
Block0304_2020_08_03_20.npy
1.0
Block0304_2020_08_03_21.npy
1.0
Block0304_2020_08_03_22.npy
1.0
Block0304_2020_08_03_23.npy
1.0
Block0304_2020_08_03_24.npy
1.0
Block0304_2020_08_03_25.npy
1.0
Block0304_2020_08_03_26.npy
1.0
Block0304_2020_08_03_27.npy
1.0
Block0304_2020_08_03_28.npy
1.0
Block0304_2020_08_03_29.npy
1.0
Block0304_2020_08_03_30.npy
1.0
Block0304_2020_08_03_31.npy
1.0
Block0304_2020_08_03_32.npy
1.0
Block0304_2020_08_03_33.npy
1.0
Block0304_2020_08_03_34.npy
1.0
Block0304_2020_08_03_35.npy
1.0
Block0304_2020_08_03_36.npy
1.0
Block0304_2020_08_03_37.npy
1.0
Block0304_2020_08_03_38.npy
1.0
Block0304_2020_08_03_39.npy
1.0
Block0304_2020_08_03_40.npy
1.0
Block030

Block 0305

In [100]:
prev_preprocessed_image_loc_0305 = "preprocessed_data/Block_0305/sub_images_and_counts"

In [101]:
folder_contents_0305 = os.listdir(prev_preprocessed_image_loc_0305)
folder_contents_0305.sort()

In [102]:
# folder_contents

In [103]:
folder_contents_0305[0]

'Block0305_2020_08_03_0.npy'

In [104]:
s_path_0305 = 'preprocessed_data/Block_0305/TS_ready_data_frames/'

In [105]:
# get the predictions for all subimages for the block? do separately for single digit subimages and double digit subimages separately
character_list = list(range(10))
character_list = list(map(str, character_list))

In [106]:
%%time

# for single digit subwindows
catch_all_dfs_0305 = []
for character in character_list:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0305, folder_contents_0305, 2, 10, character, s_path_0305)
    catch_all_dfs_0305.append(features_df)

Block0305_2020_08_03_0.npy
1.0
Block0305_2020_08_03_1.npy
1.0
Block0305_2020_08_03_2.npy
1.0
Block0305_2020_08_03_3.npy
1.0
Block0305_2020_08_03_4.npy
1.0
Block0305_2020_08_03_5.npy
1.0
Block0305_2020_08_03_6.npy
1.0
Block0305_2020_08_03_7.npy
1.0
Block0305_2020_08_03_8.npy
1.0
Block0305_2020_08_03_9.npy
1.0
CPU times: user 11.8 s, sys: 1.14 s, total: 12.9 s
Wall time: 30 s


In [107]:
# get the predictions for double digits
character_list_double = list(range(10, 42))
character_list_double = list(map(str, character_list_double))

In [108]:
%%time

# for double digit subwindows
catch_all_dfs_0305 = []
for character in character_list_double:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0305, folder_contents_0305, 3, 11, character, s_path_0305)
    catch_all_dfs_0305.append(features_df)

Block0305_2020_08_03_10.npy
1.0
Block0305_2020_08_03_11.npy
1.0
Block0305_2020_08_03_12.npy
1.0
Block0305_2020_08_03_13.npy
1.0
Block0305_2020_08_03_14.npy
1.0
Block0305_2020_08_03_15.npy
1.0
Block0305_2020_08_03_16.npy
1.0
Block0305_2020_08_03_17.npy
1.0
Block0305_2020_08_03_18.npy
1.0
Block0305_2020_08_03_19.npy
1.0
Block0305_2020_08_03_20.npy
1.0
Block0305_2020_08_03_21.npy
1.0
Block0305_2020_08_03_22.npy
1.0
Block0305_2020_08_03_23.npy
1.0
Block0305_2020_08_03_24.npy
1.0
Block0305_2020_08_03_25.npy
1.0
Block0305_2020_08_03_26.npy
1.0
Block0305_2020_08_03_27.npy
1.0
Block0305_2020_08_03_28.npy
1.0
Block0305_2020_08_03_29.npy
1.0
Block0305_2020_08_03_30.npy
1.0
Block0305_2020_08_03_31.npy
1.0
Block0305_2020_08_03_32.npy
1.0
Block0305_2020_08_03_33.npy
1.0
Block0305_2020_08_03_34.npy
1.0
Block0305_2020_08_03_35.npy
1.0
Block0305_2020_08_03_36.npy
1.0
Block0305_2020_08_03_37.npy
1.0
Block0305_2020_08_03_38.npy
1.0
Block0305_2020_08_03_39.npy
1.0
Block0305_2020_08_03_40.npy
1.0
Block030

Block 0306

In [109]:
prev_preprocessed_image_loc_0306 = "preprocessed_data/Block_0306/sub_images_and_counts"

In [110]:
folder_contents_0306 = os.listdir(prev_preprocessed_image_loc_0306)
folder_contents_0306.sort()

In [111]:
# folder_contents

In [112]:
folder_contents_0306[0]

'Block0306_2020_08_03_0.npy'

In [113]:
s_path_0306 = 'preprocessed_data/Block_0306/TS_ready_data_frames/'

In [114]:
# get the predictions for all subimages for the block? do separately for single digit subimages and double digit subimages separately
character_list = list(range(10))
character_list = list(map(str, character_list))

In [115]:
%%time

# for single digit subwindows
catch_all_dfs_0306 = []
for character in character_list:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0306, folder_contents_0306, 2, 10, character, s_path_0306)
    catch_all_dfs_0306.append(features_df)

Block0306_2020_08_03_0.npy
1.0
Block0306_2020_08_03_1.npy
1.0
Block0306_2020_08_03_2.npy
1.0
Block0306_2020_08_03_3.npy
1.0
Block0306_2020_08_03_4.npy
1.0
Block0306_2020_08_03_5.npy
1.0
Block0306_2020_08_03_6.npy
1.0
Block0306_2020_08_03_7.npy
1.0
Block0306_2020_08_03_8.npy
1.0
Block0306_2020_08_03_9.npy
1.0
CPU times: user 11.8 s, sys: 1.15 s, total: 12.9 s
Wall time: 32.4 s


In [116]:
# get the predictions for double digits
character_list_double = list(range(10, 42))
character_list_double = list(map(str, character_list_double))

In [117]:
%%time

# for double digit subwindows
catch_all_dfs_0306 = []
for character in character_list_double:
    features_df = extract_features_and_get_counts(feature_extractor_model, prev_preprocessed_image_loc_0306, folder_contents_0306, 3, 11, character, s_path_0306)
    catch_all_dfs_0306.append(features_df)

Block0306_2020_08_03_10.npy
1.0
Block0306_2020_08_03_11.npy
1.0
Block0306_2020_08_03_12.npy
1.0
Block0306_2020_08_03_13.npy
1.0
Block0306_2020_08_03_14.npy
1.0
Block0306_2020_08_03_15.npy
1.0
Block0306_2020_08_03_16.npy
1.0
Block0306_2020_08_03_17.npy
1.0
Block0306_2020_08_03_18.npy
1.0
Block0306_2020_08_03_19.npy
1.0
Block0306_2020_08_03_20.npy
1.0
Block0306_2020_08_03_21.npy
1.0
Block0306_2020_08_03_22.npy
1.0
Block0306_2020_08_03_23.npy
1.0
Block0306_2020_08_03_24.npy
1.0
Block0306_2020_08_03_25.npy
1.0
Block0306_2020_08_03_26.npy
1.0
Block0306_2020_08_03_27.npy
1.0
Block0306_2020_08_03_28.npy
1.0
Block0306_2020_08_03_29.npy
1.0
Block0306_2020_08_03_30.npy
1.0
Block0306_2020_08_03_31.npy
1.0
Block0306_2020_08_03_32.npy
1.0
Block0306_2020_08_03_33.npy
1.0
Block0306_2020_08_03_34.npy
1.0
Block0306_2020_08_03_35.npy
1.0
Block0306_2020_08_03_36.npy
1.0
Block0306_2020_08_03_37.npy
1.0
Block0306_2020_08_03_38.npy
1.0
Block0306_2020_08_03_39.npy
1.0
Block0306_2020_08_03_40.npy
1.0
Block030